In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from tensorflow.keras import layers
from sklearn.metrics import f1_score
import os

In [3]:
# Load the dataset
data_path = '/Users/as274094/Documents/psf_dataset1/'
# data_path = '/gpfswork/rech/prk/uzk69cg/psf_dataset1/'
dataset = np.load(data_path + 'PCA_dataset1.npy', allow_pickle=True)[()]

x_train = dataset['train_stars_pca']
x_val = dataset['validation_stars_pca']
x_test = dataset['test_stars_pca']
y_train = dataset['train_C']
y_val = dataset['validation_C']
y_test = dataset['test_C']
SED_test = dataset['test_SEDs']


In [4]:
# Hyperparameters
PCA_components = 24
model_learning_rate = 0.1
N_epochs = 10
N_committee = 48
patience_epochs = 3

In [5]:
def SEDlisttoC(SED_list):
    sed_array = np.array(SED_list)
    return sed_array*0.5 + 1.5

def CtoSEDarray(c_values, variance):
    sed_classes = ((c_values - 1.25) // 0.5).astype(int)
    sed_classes = np.where((c_values < 1.25) | (c_values > 7.75), 20, sed_classes)
    sed_classes = np.where((variance > 1.00), 20, sed_classes)
    return sed_classes

def calculate_success_rate(confusion_matrix):
    diagonal = np.trace(confusion_matrix)
    diagonal_neighbors = np.sum(np.diagonal(confusion_matrix, offset=1)) + np.sum(np.diagonal(confusion_matrix, offset=-1))
    total_classified = np.sum(confusion_matrix)
    
    success_rate = (diagonal + diagonal_neighbors) / total_classified
    return success_rate

In [6]:
# Define the model architecture

initializer = tf.keras.initializers.GlorotNormal(seed = 1)
model = tf.keras.Sequential([
    layers.Dense(26, input_shape=[PCA_components], activation='sigmoid', kernel_initializer= initializer),
    layers.Dense(26, activation='sigmoid', kernel_initializer= initializer),
    layers.Dense(1, activation = 'linear', kernel_initializer= initializer)
])
model.compile(
    loss = tf.keras.losses.MeanSquaredError(),
    optimizer = tf.keras.optimizers.Adam(learning_rate = model_learning_rate)
)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", patience = patience_epochs, restore_best_weights=True)
progbar_logger = tf.keras.callbacks.ProgbarLogger()

start_time = time.time() # Measure training time
learning = model.fit(x_train, y_train, epochs= N_epochs, callbacks = [progbar_logger,early_stopping], validation_data=(x_val,y_val)) # Train the model

# Calculate the training time
end_time = time.time()
training_time = end_time - start_time
print("Training time:", training_time, "seconds")


Epoch 1/10
1000/1000 [==============================] - 4s 4ms/sample - loss: 3.6541 - val_loss: 3.8253
Epoch 2/10
   0/1000 [..............................] - ETA: 0s - loss: 3.6366

KeyboardInterrupt: 

In [7]:
# Load a trained model

model_path = '/Users/as274094/GitHub/Refractored_star_classifier/tensorflow_version/single_model/my_model.h5'
my_model = tf.keras.models.load_model(model_path)

In [8]:
# Make predictions

C_pred = model.predict(x_test, verbose = 1).reshape(-1) # Predict the scalar parameter C
SED_pred = CtoSEDarray(C_pred,np.zeros_like(C_pred))

625/625 [==============================] - 1s 1ms/step


In [9]:
# Evaluate the performance of the classifier

mse = np.mean((y_test - C_pred)**2)
print('MSE:', mse)

f1 = f1_score(SED_test, SED_pred, average = None)
f1_mean = np.mean(f1[:13])
print('\nF1 score for each class:', f1)
print('Average F1 score:', f1_mean)

confusion_matrix = tf.math.confusion_matrix(SED_test, SED_pred) 
print("\nConfusion matrix:")
print(confusion_matrix)

success_rate = calculate_success_rate(confusion_matrix)
print('\nSuccess rate:', success_rate)


MSE: 3.5137728569869933

F1 score for each class: [0.         0.         0.         0.         0.         0.
 0.14255863 0.         0.         0.         0.         0.
 0.        ]
Average F1 score: 0.010966048114875605

Confusion matrix:
tf.Tensor(
[[   0    0    0    0    0    0 1520    0    0    0    0    0    0]
 [   0    0    0    0    0    0 1526    0    0    0    0    0    0]
 [   0    0    0    0    0    0 1570    0    0    0    0    0    0]
 [   0    0    0    0    0    0 1579    0    0    0    0    0    0]
 [   0    0    0    0    0    0 1515    0    0    0    0    0    0]
 [   0    0    0    0    0    0 1535    0    0    0    0    0    0]
 [   0    0    0    0    0    0 1535    0    0    0    0    0    0]
 [   0    0    0    0    0    0 1569    0    0    0    0    0    0]
 [   0    0    0    0    0    0 1479    0    0    0    0    0    0]
 [   0    0    0    0    0    0 1555    0    0    0    0    0    0]
 [   0    0    0    0    0    0 1525    0    0    0    0    0    0]
 [

In [ ]:
#Save the committee models

os.makedirs("single_model", exist_ok=True)
model.save(f"single_model/my_model.h5")

In [ ]:
# PLot the loss function evolution

loss_evolution = learning.history["loss"]
val_loss_evolution = learning.history["val_loss"]

plt.plot(loss_evolution,label = "Train set")
plt.plot(val_loss_evolution,label = "Validation set")
plt.xlabel("Epochs")
plt.ylabel("Loss function value")
plt.legend()
plt.title("Loss function evolution")


In [ ]:
# Plot the metrics

plt.figure(figsize = (9,5))
plt.bar(np.arange(13), height = f1, tick_label = np.arange(13) +1 ,label = "F1 score")
plt.axhline(f1_mean, color='red', linestyle='--', label = 'F1 score average')
plt.axhline(success_rate, color='purple', label = 'Success rate')
plt.xlabel("Spectral class")
plt.ylabel("Metric")
plt.legend()